In [9]:
import nltk
import numpy as np
from json import load
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from google.colab.files import upload
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Preprocessing metods are defined like stemming and Tokenization. Bag of words is defined to convert in vector form

In [10]:
stemmer=PorterStemmer()
def tokenize(sentence):
  return nltk.word_tokenize(sentence)
def stem(word):
  return stemmer.stem(word.lower())
def bag_of_words(tokenized_sentence,all_words):
  tokenized_sentence=[stem(w) for w in tokenized_sentence]
  bag=np.zeros(len(all_words),dtype=np.float32)
  for idx,w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx]=1.0
  return bag

## Loading of Json Data and creation of Traing Data

In [11]:
with open('intents.json','r') as f:
  intents=load(f)
all_words=[]
tags=[]
xy=[]
for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w=tokenize(pattern)
    all_words.extend(w)
    xy.append((w,tag))
ignore_words=["?","!",",","."]
all_words=sorted(set([stem(w) for w in all_words if w not in ignore_words]))
x_train,y_train=[],[]
for (pattern_sentence,tag) in xy:
  bag=bag_of_words(pattern_sentence,all_words)
  x_train.append(bag)
  label=tags.index(tag)
  y_train.append(label)
x_train=np.array(x_train)
y_train=np.array(y_train)

## Data converted to batches for easy acces during training of Model using Dataset and DataLoader

In [12]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples=len(x_train)
    self.x_data=x_train
    self.y_data=y_train
  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]
  def __len__(self):
    return self.n_samples

batch_size=8
hidden_size=8
output_size=len(tags)
input_size=len(all_words)
dataset=ChatDataset()
train_loader=DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True)

## Model building with desired architecture

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(NeuralNet,self).__init__()
    self.l1=nn.Linear(input_size,hidden_size)
    self.l2=nn.Linear(hidden_size,hidden_size)
    self.l3=nn.Linear(hidden_size,num_classes)
    self.relu=nn.ReLU()
  def forward(self,x):
    out=self.relu(self.l1(x))
    out=self.relu(self.l2(out))
    out=self.l3(out)
    return out

## Training of the Model.

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=NeuralNet(input_size,hidden_size,output_size).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
for epoch in range(1000):
  for (words,labels) in train_loader:
    words=words.to(device)
    labels=labels.to(device)
    outputs=model(words)
    loss=criterion(outputs,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  if epoch%100==0:
    print("epoch={} and loss={}".format(epoch,loss.item()))
print("final loss=",loss)

epoch=0 and loss=1.74021315574646
epoch=100 and loss=0.8956341743469238
epoch=200 and loss=0.21258723735809326
epoch=300 and loss=0.04612146317958832
epoch=400 and loss=0.02971796505153179
epoch=500 and loss=0.009276503697037697
epoch=600 and loss=0.004896452650427818
epoch=700 and loss=0.0009313729824498296
epoch=800 and loss=0.001419661333784461
epoch=900 and loss=0.00484789814800024
final loss= tensor(0.0006, grad_fn=<NllLossBackward>)


## Saving The trained model parameters and architecture of the model along with word vocabulary and tags.

## Running and testing the trained Model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
bot_name="RG"
print("Let's chat: type 'quit' to exit")
while True:
  sentence=input("You:")
  if sentence=="quit":
    break
  sentence=tokenize(sentence)
  x=bag_of_words(sentence,all_words)
  x=x.reshape(1,x.shape[0])
  x=torch.from_numpy(x)
  output=model(x)
  _,predicted=torch.max(output,dim=1)
  tag=tags[predicted.item()]
  probs=torch.softmax(output,dim=1)
  prob=probs[0][predicted.item()]
  if prob.item()>0.75:
    for intent in intents["intents"]:
      if tag==intent["tag"]:
        response=np.random.choice(intent["responses"])
        print("{}: {}".format(bot_name,response))
    if tag=="goodbye":
      break
  else:
    print("{}: I do not understand...".format(bot_name))

Let's chat: type 'quit' to exit
You:hi
RG: Hey :-)
You:when can you deliver
RG: I do not understand...
You:how long it takes you to deliver
RG: Delivery takes 2-4 days
You:how can i pay you
RG: I do not understand...
You:do you take cash
RG: We accept most major credit cards, and Paypal
You:ok bye
RG: Have a nice day
